In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [13]:
#tail -n +2 train.csv | split -l 150000


In [14]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [5]:
path = 'train/%s' % (np.random.choice(splits))
#
columns = ['acoustic_data','time_to_failure']
df = pd.read_csv(path, float_precision='round_trip', header=None)
df.columns = columns

df.describe()

,acoustic_data,time_to_failure
count,150000.000000,150000.000000
mean,4.182680,0.637582
std,6.946378,0.011254
min,-188.000000,0.617600
25%,2.000000,0.628296
50%,4.000000,0.637797
75%,7.000000,0.647397
max,219.000000,0.656998


In [10]:
np.fft.fft(df['acoustic_data'])[0]

(1254804+0j)

In [30]:
df['roll_1000'] = df['acoustic_data'].rolling(1000,min_periods=1000).mean()
df['roll_1000'].fillna(df['roll_1000'][1000],inplace=True)
df['shifted'] = df['roll_1000'].shift(1)
df['shifted'].fillna(df['roll_1000'][1000],inplace=True)
df["roll_diff"] = df['shifted'] - df['roll_1000']

In [32]:
df.sample(10)

,acoustic_data,time_to_failure,roll_1000,shifted,roll_diff
40751,2,5.280099,4.010,4.013,0.003
133457,9,5.256696,4.420,4.411,-0.009
87705,-6,5.268397,3.851,3.859,0.008
1073,1,5.290797,3.881,3.883,0.002
39585,4,5.281196,3.296,3.293,-0.003
67038,-2,5.273797,3.371,3.375,0.004
94815,4,5.266298,4.023,4.024,0.001
110551,0,5.262099,4.158,4.162,0.004
65101,10,5.273799,3.303,3.293,-0.010
73439,6,5.271599,3.609,3.610,0.001


In [15]:
TIMESTEPS=150

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

param_test = False
param_augment = False
param_scale = True
param_noise = 0.3
param_smart_augment = False


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,
                                                  param_scale,param_noise,param_smart_augment]) for chunk in split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [22]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

res[0].get()[100].head()

,acoustic_data_median,acoustic_data_mean,acoustic_data_max,acoustic_data_min,acoustic_data_std,acoustic_data_abs,acoustic_data_q25,acoustic_data_q50,acoustic_data_q75,acoustic_data_sum,acoustic_data_uniq,acoustic_data_pos,acoustic_data_negs,acoustic_data_ssum,acoustic_data_imax,acoustic_data_imin,acoustic_data_abs_nrg,acoustic_data_abs_sum_chg,acoustic_data_autocorr_10,acoustic_data_cid_ce,acoustic_data_kurtosis,acoustic_data_mean_chg,acoustic_data_reocurring_pct,acoustic_data_r_sigma,acoustic_data_ratio_to_length,acoustic_data_skewness,acoustic_data_perc_reocurr_dp,acoustic_data_perc_reocurr_all,acoustic_data_sum_reoccurr_val,acoustic_data_sum_reoccurr_dp,acoustic_data_mean_head,acoustic_data_mean_tail,acoustic_data_abs_diff_head_tail,acoustic_data_pos_head,acoustic_data_neg_head,acoustic_data_pos_tail,acoustic_data_neg_tail,roll_diff_1_median,roll_diff_1_mean,roll_diff_1_max,roll_diff_1_min,roll_diff_1_std,roll_diff_1_abs,roll_diff_1_q25,roll_diff_1_q50,roll_diff_1_q75,roll_diff_1_sum,roll_diff_1_uniq,roll_diff_1_pos,roll_diff_1_negs,roll_diff_1_ssum,roll_diff_1_imax,roll_diff_1_imin,roll_diff_1_abs_nrg,roll_diff_1_abs_sum_chg,roll_diff_1_autocorr_10,roll_diff_1_cid_ce,roll_diff_1_kurtosis,roll_diff_1_mean_chg,roll_diff_1_reocurring_pct,roll_diff_1_r_sigma,roll_diff_1_ratio_to_length,roll_diff_1_skewness,roll_diff_1_perc_reocurr_dp,roll_diff_1_perc_reocurr_all,roll_diff_1_sum_reoccurr_val,roll_diff_1_sum_reoccurr_dp,roll_diff_1_mean_head,roll_diff_1_mean_tail,roll_diff_1_abs_diff_head_tail,roll_diff_1_pos_head,roll_diff_1_neg_head,roll_diff_1_pos_tail,roll_diff_1_neg_tail,time_to_failure,augmented
0,1.15939,1.460069,0.609829,-0.627335,0.934842,0.620113,-2.070432,1.15939,2.625769,1.460069,1.144822,-2.505708,2.614461,1.120430,0.193095,0.713059,0.198884,0.690363,-0.759203,-2.184708,-0.062220,0.663884,0.151163,0.510355,1.144822,-0.642596,0.151163,-0.102828,1.013737,1.251579,1.460069,1.460069,0.620113,-2.505708,2.614461,-2.505708,2.614461,-0.058655,0.007603,-0.654757,0.773016,-0.968219,-0.715845,1.680780,-0.058655,-1.473800,0.007603,-1.530082,-10.280173,-10.382073,-1.185432,1.988762,-1.698393,-0.271050,-3.264399,1.439552,0.575157,12.204346,0.196861,-3.670028,-5.099089,-1.530082,12.192913,-3.670028,0.640621,-0.017802,-0.002662,0.007603,0.007603,-0.715845,-10.280173,-10.382073,-10.280173,-10.382073,3.150997,False
1,1.15939,2.359906,0.080369,-0.002862,0.171982,0.040879,-0.135997,1.15939,1.330692,2.359906,0.294105,-1.072772,1.005117,0.279295,-0.806026,-0.901621,-0.049595,-0.060469,0.146688,-1.351421,-0.248531,-0.187978,-0.233956,0.791800,0.294105,-0.026025,-0.233956,-0.013152,1.443496,2.500382,2.359906,2.359906,0.040879,-1.072772,1.005117,-1.072772,1.005117,-0.058655,-1.056813,0.134107,-0.379532,0.584239,0.257542,-1.384472,-0.058655,1.222175,-1.056813,0.727641,0.369615,0.811639,0.832075,1.525535,0.482119,0.067886,0.384085,-0.287661,-1.773210,-0.089220,0.494634,0.134234,-0.721730,0.727641,-0.130198,0.134234,-0.078505,-0.671160,-0.582242,-1.056813,-1.056813,0.257542,0.369615,0.811639,0.369615,0.811639,3.150997,False
2,1.15939,0.771732,-0.163997,0.231315,-0.170203,-0.198804,0.039861,1.15939,0.035615,0.771732,-0.232529,-0.020935,-0.031671,-0.121601,0.831708,-0.898200,-0.140535,-0.237892,1.603408,-0.247671,-0.330894,-0.528722,0.395637,0.135094,-0.232529,-0.705836,0.395637,0.255874,-0.156986,0.831561,0.771732,0.771732,-0.198804,-0.020935,-0.031671,-0.020935,-0.031671,2.874077,1.836789,-0.169302,0.227072,-0.078553,-0.198733,-0.070793,2.874077,0.451897,1.836789,-0.047739,1.115778,-0.777732,-0.013001,-0.769052,0.828349,-0.157344,-0.140083,0.595684,-0.658480,-0.090099,0.196861,0.010505,0.452683,-0.047739,-0.118819,0.010505,0.146222,-0.273464,0.798766,1.836789,1.836789,-0.198733,1.115778,-0.777732,1.115778,-0.777732,3.150997,False
3,1.15939,0.737123,-0.326907,0.348404,-0.342476,-0.338619,0.743292,1.15939,0.035615,0.737123,-0.435081,0.893705,-0.805394,-0.251824,-1.094432,-0.200330,-0.167070,-0.356174,1.040576,0.680410,-0.509064,-0.017605,-0.5105

In [8]:

import pickle

shared_train_data = pickle.load(open("shared_train_data.pickle", "rb"))
shared_val_data = pickle.load(open("shared_val_data.pickle", "rb"))

In [23]:
features = ['acoustic_data_mean', 'acoustic_data_std', 'acoustic_data_min', 'acoustic_data_max',\
           'roll_diff_1_mean', 'roll_diff_1_std', 'roll_diff_1_min', 'roll_diff_1_max']
NUMBER_OF_FEATURES=len(features)

# training = np.empty((len(splits),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# targets = np.empty((len(splits),1),dtype=float)
# train_data = np.empty((len(shared_train_data),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# y_train = np.empty((len(shared_train_data),1),dtype=float)
# val_data = np.empty((len(shared_val_data),TIMESTEPS,NUMBER_OF_FEATURES),dtype=float)
# y_val = np.empty((len(shared_val_data),1),dtype=float)



training = []
targets = []
val_data = []
y_val = []


for r in res:
    for df in r.get():
#         if df['segment'].unique() in shared_train_data:
            training.append(df[features].values)
            targets.append(np.array([df['time_to_failure'].unique()]))
#         elif df['segment'].unique() in shared_val_data: 
#             val_data.append(df.drop(['time_to_failure','segment','augmented'], axis=1).values)
#             y_val.append(np.array([df['time_to_failure'].unique()]))
            
training = np.array(training).reshape(-1,TIMESTEPS,NUMBER_OF_FEATURES)
targets = np.array(targets).reshape((len(targets),1))
# val_data = np.array(val_data).reshape((len(shared_val_data),TIMESTEPS,NUMBER_OF_FEATURES))
# y_val = np.array(y_val).reshape((len(shared_val_data),1))

In [24]:
y_val

[]

In [34]:
train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint


dropout=0.3
BATCH_SIZE=128


my_model = Sequential()


my_model.add(GRU(#use_bias = True,#unit_forget_bias=True,\
                  units = 64,\
                  input_shape=(TIMESTEPS, NUMBER_OF_FEATURES),
                  dropout=dropout,recurrent_dropout=dropout,
                  #return_sequences=True
                  ))

#my_model.add(BatchNormalization())


                 
                    
my_model.add(Dense(10,activation='relu'))
my_model.add(BatchNormalization())
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()

from keras.callbacks import Callback

filepath="lstm.weights.{epoch:02d}-{val_loss:.2f}.hdf5"


callbacks = [
    #EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
]


#my_model.load_weights('lstm.weights.393-2.15.hdf5')

history = my_model.fit(train_data, y_train, batch_size=BATCH_SIZE, epochs=1000, shuffle=True,
                      validation_data=(val_data,y_val), callbacks=callbacks
                    )


# import matplotlib.pyplot as plt


# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, 64)                14016     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
_________________________________________________________________
batch_normalization_3 (Batch (None, 10)                40        
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 11        
Total params: 14,717
Trainable params: 14,697
Non-trainable params: 20
_________________________________________________________________
Train on 3774 samples, validate on 420 samples
Epoch 1/1000
3774/3774 [==============================] - 7s 2ms/step - loss: 5.6709 - mean_absolute_error: 5.6709 - val_loss: 5.2673 - val_mean_absolute_error: 5.2673

Epoch 00001: val_loss improved from inf to 5.26729, s

3774/3774 [==============================] - 5s 1ms/step - loss: 2.9415 - mean_absolute_error: 2.9415 - val_loss: 2.9551 - val_mean_absolute_error: 2.9551

Epoch 00030: val_loss improved from 2.95841 to 2.95514, saving model to lstm.weights.30-2.96.hdf5
Epoch 31/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.9686 - mean_absolute_error: 2.9686 - val_loss: 2.9649 - val_mean_absolute_error: 2.9649

Epoch 00031: val_loss did not improve from 2.95514
Epoch 32/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.9281 - mean_absolute_error: 2.9281 - val_loss: 2.9572 - val_mean_absolute_error: 2.9572

Epoch 00032: val_loss did not improve from 2.95514
Epoch 33/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.9480 - mean_absolute_error: 2.9480 - val_loss: 2.9662 - val_mean_absolute_error: 2.9662

Epoch 00033: val_loss did not improve from 2.95514
Epoch 34/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.93

3774/3774 [==============================] - 5s 1ms/step - loss: 2.8239 - mean_absolute_error: 2.8239 - val_loss: 2.8130 - val_mean_absolute_error: 2.8130

Epoch 00065: val_loss improved from 2.83251 to 2.81296, saving model to lstm.weights.65-2.81.hdf5
Epoch 66/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.8168 - mean_absolute_error: 2.8168 - val_loss: 2.8273 - val_mean_absolute_error: 2.8273

Epoch 00066: val_loss did not improve from 2.81296
Epoch 67/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.8137 - mean_absolute_error: 2.8137 - val_loss: 2.7939 - val_mean_absolute_error: 2.7939

Epoch 00067: val_loss improved from 2.81296 to 2.79389, saving model to lstm.weights.67-2.79.hdf5
Epoch 68/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.7945 - mean_absolute_error: 2.7945 - val_loss: 2.7583 - val_mean_absolute_error: 2.7583

Epoch 00068: val_loss improved from 2.79389 to 2.75834, saving model to lstm.weights.68

3774/3774 [==============================] - 5s 1ms/step - loss: 2.6436 - mean_absolute_error: 2.6436 - val_loss: 2.4982 - val_mean_absolute_error: 2.4982

Epoch 00099: val_loss did not improve from 2.48560
Epoch 100/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.6170 - mean_absolute_error: 2.6170 - val_loss: 2.4811 - val_mean_absolute_error: 2.4811

Epoch 00100: val_loss improved from 2.48560 to 2.48109, saving model to lstm.weights.100-2.48.hdf5
Epoch 101/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.6311 - mean_absolute_error: 2.6311 - val_loss: 2.4631 - val_mean_absolute_error: 2.4631

Epoch 00101: val_loss improved from 2.48109 to 2.46308, saving model to lstm.weights.101-2.46.hdf5
Epoch 102/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.6209 - mean_absolute_error: 2.6209 - val_loss: 2.5674 - val_mean_absolute_error: 2.5674

Epoch 00102: val_loss did not improve from 2.46308
Epoch 103/1000
3774/3774 [=====

3774/3774 [==============================] - 5s 1ms/step - loss: 2.5359 - mean_absolute_error: 2.5359 - val_loss: 2.4446 - val_mean_absolute_error: 2.4446

Epoch 00134: val_loss did not improve from 2.41815
Epoch 135/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.5442 - mean_absolute_error: 2.5442 - val_loss: 2.4477 - val_mean_absolute_error: 2.4477

Epoch 00135: val_loss did not improve from 2.41815
Epoch 136/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.5223 - mean_absolute_error: 2.5223 - val_loss: 2.4742 - val_mean_absolute_error: 2.4742

Epoch 00136: val_loss did not improve from 2.41815
Epoch 137/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.4996 - mean_absolute_error: 2.4996 - val_loss: 2.4325 - val_mean_absolute_error: 2.4325

Epoch 00137: val_loss did not improve from 2.41815
Epoch 138/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.5069 - mean_absolute_error: 2.5069 - val_loss

...

3774/3774 [==============================] - 5s 1ms/step - loss: 2.0264 - mean_absolute_error: 2.0264 - val_loss: 2.4118 - val_mean_absolute_error: 2.4118

Epoch 00834: val_loss did not improve from 2.34089
Epoch 835/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0516 - mean_absolute_error: 2.0516 - val_loss: 2.4069 - val_mean_absolute_error: 2.4069

Epoch 00835: val_loss did not improve from 2.34089
Epoch 836/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0176 - mean_absolute_error: 2.0176 - val_loss: 2.3750 - val_mean_absolute_error: 2.3750

Epoch 00836: val_loss did not improve from 2.34089
Epoch 837/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0251 - mean_absolute_error: 2.0251 - val_loss: 2.3884 - val_mean_absolute_error: 2.3884

Epoch 00837: val_loss did not improve from 2.34089
Epoch 838/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0448 - mean_absolute_error: 2.0448 - val_loss

3774/3774 [==============================] - 5s 1ms/step - loss: 2.0417 - mean_absolute_error: 2.0417 - val_loss: 2.4114 - val_mean_absolute_error: 2.4114

Epoch 00871: val_loss did not improve from 2.34089
Epoch 872/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0684 - mean_absolute_error: 2.0684 - val_loss: 2.4029 - val_mean_absolute_error: 2.4029

Epoch 00872: val_loss did not improve from 2.34089
Epoch 873/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0209 - mean_absolute_error: 2.0209 - val_loss: 2.3910 - val_mean_absolute_error: 2.3910

Epoch 00873: val_loss did not improve from 2.34089
Epoch 874/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9987 - mean_absolute_error: 1.9987 - val_loss: 2.4044 - val_mean_absolute_error: 2.4044

Epoch 00874: val_loss did not improve from 2.34089
Epoch 875/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0279 - mean_absolute_error: 2.0279 - val_loss

3774/3774 [==============================] - 5s 1ms/step - loss: 2.0213 - mean_absolute_error: 2.0213 - val_loss: 2.4015 - val_mean_absolute_error: 2.4015

Epoch 00908: val_loss did not improve from 2.34089
Epoch 909/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0129 - mean_absolute_error: 2.0129 - val_loss: 2.4103 - val_mean_absolute_error: 2.4103

Epoch 00909: val_loss did not improve from 2.34089
Epoch 910/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9968 - mean_absolute_error: 1.9968 - val_loss: 2.4105 - val_mean_absolute_error: 2.4105

Epoch 00910: val_loss did not improve from 2.34089
Epoch 911/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0396 - mean_absolute_error: 2.0396 - val_loss: 2.4054 - val_mean_absolute_error: 2.4054

Epoch 00911: val_loss did not improve from 2.34089
Epoch 912/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9702 - mean_absolute_error: 1.9702 - val_loss

3774/3774 [==============================] - 5s 1ms/step - loss: 1.9913 - mean_absolute_error: 1.9913 - val_loss: 2.3951 - val_mean_absolute_error: 2.3951

Epoch 00945: val_loss did not improve from 2.34089
Epoch 946/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9893 - mean_absolute_error: 1.9893 - val_loss: 2.4257 - val_mean_absolute_error: 2.4257

Epoch 00946: val_loss did not improve from 2.34089
Epoch 947/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0048 - mean_absolute_error: 2.0048 - val_loss: 2.4100 - val_mean_absolute_error: 2.4100

Epoch 00947: val_loss did not improve from 2.34089
Epoch 948/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9954 - mean_absolute_error: 1.9954 - val_loss: 2.4032 - val_mean_absolute_error: 2.4032

Epoch 00948: val_loss did not improve from 2.34089
Epoch 949/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0329 - mean_absolute_error: 2.0329 - val_loss

3774/3774 [==============================] - 5s 1ms/step - loss: 2.0199 - mean_absolute_error: 2.0199 - val_loss: 2.4132 - val_mean_absolute_error: 2.4132

Epoch 00982: val_loss did not improve from 2.34089
Epoch 983/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0118 - mean_absolute_error: 2.0118 - val_loss: 2.3924 - val_mean_absolute_error: 2.3924

Epoch 00983: val_loss did not improve from 2.34089
Epoch 984/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9959 - mean_absolute_error: 1.9959 - val_loss: 2.3852 - val_mean_absolute_error: 2.3852

Epoch 00984: val_loss did not improve from 2.34089
Epoch 985/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 2.0045 - mean_absolute_error: 2.0045 - val_loss: 2.4305 - val_mean_absolute_error: 2.4305

Epoch 00985: val_loss did not improve from 2.34089
Epoch 986/1000
3774/3774 [==============================] - 5s 1ms/step - loss: 1.9882 - mean_absolute_error: 1.9882 - val_loss

In [14]:

columns=res[0].get()[0].columns.values

data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        data = data.append(df)
        i+=1
        

data.reset_index(drop=True,inplace=True)




In [15]:
preds = []

grouped = data[data['segment'].isin(shared_val_data)]\
        .drop(['time_to_failure', 'augmented'], axis=1)\
        .sort_values(by='segment').groupby('segment')


for name, group in grouped:
    preds.append(my_model.predict(group.drop(['segment'], axis=1).values.reshape(1,TIMESTEPS,NUMBER_OF_FEATURES))[0][0])

In [16]:
pickle.dump(preds, open( "stacked_lstm_val.pickle", "wb" ))


In [18]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [19]:
test_splits

['seg_b691c5.csv',
 'seg_a8e71a.csv',
 'seg_355717.csv',
 'seg_4e627b.csv',
 'seg_3fb24b.csv',
 'seg_0d0250.csv',
 'seg_7f3ab0.csv',
 'seg_272a47.csv',
 'seg_750e7e.csv',
 'seg_75dc3e.csv',
 'seg_c35940.csv',
 'seg_cd43b4.csv',
 'seg_7d88e3.csv',
 'seg_bdae02.csv',
 'seg_9c3ef8.csv',
 'seg_1bf7b3.csv',
 'seg_0dc38f.csv',
 'seg_bff392.csv',
 'seg_50b048.csv',
 'seg_507065.csv',
 'seg_f0db55.csv',
 'seg_0cdcc8.csv',
 'seg_687a5c.csv',
 'seg_566efe.csv',
 'seg_00e5f7.csv',
 'seg_a25c46.csv',
 'seg_8e5704.csv',
 'seg_6e572a.csv',
 'seg_5f352e.csv',
 'seg_a802fa.csv',
 'seg_cd2b34.csv',
 'seg_280e3b.csv',
 'seg_f383fb.csv',
 'seg_a56c04.csv',
 'seg_3f3983.csv',
 'seg_d516e3.csv',
 'seg_50e270.csv',
 'seg_170684.csv',
 'seg_f5d2dd.csv',
 'seg_ce9f59.csv',
 'seg_11647f.csv',
 'seg_2c69e4.csv',
 'seg_3452b2.csv',
 'seg_9bab11.csv',
 'seg_f8db0a.csv',
 'seg_01c775.csv',
 'seg_8cdf83.csv',
 'seg_b21273.csv',
 'seg_2ed79b.csv',
 'seg_81c1a6.csv',
 'seg_266805.csv',
 'seg_19e1ff.csv',
 'seg_259d65

In [20]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    test_res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
300
300
300
300
300
300
300
300


In [22]:
ids = []
preds = []
i=0
for r in test_res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        #print(test_df.values.shape)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600


In [25]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('lstm_preds.csv', index=False)

In [24]:
submission["time_to_failure"].describe()

count    2624.000000
mean     5.397872   
std      2.745490   
min     -0.016591   
25%      3.075488   
50%      4.849690   
75%      7.773914   
max      11.540263  
Name: time_to_failure, dtype: float64

In [52]:
submission["time_to_failure"].describe()

count    2624.000000
mean     5.322992   
std      2.626039   
min     -0.497457   
25%      3.176856   
50%      4.755400   
75%      7.700604   
max      10.682412  
Name: time_to_failure, dtype: float64

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()